## Dependencies

---

In [1]:
import yaml
import re
from datetime import datetime
import pandas as pd
from pandas import DataFrame

In [2]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.edge.service import Service
from webdriver_manager.microsoft import EdgeChromiumDriverManager
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC

In [3]:
from selenium.common.exceptions import NoSuchElementException
from selenium.common.exceptions import TimeoutException

In [4]:
# edge:object = webdriver.Edge(executable_path="./utils/msedgedriver.exe")
edge = webdriver.Edge(service=Service(EdgeChromiumDriverManager().install()))
edge.maximize_window()

[WDM] - Downloading: 100%|██████████| 8.22M/8.22M [00:01<00:00, 4.91MB/s]


### Functions:

In [5]:
def resources(name:str) -> dict[str:str]:
    """Read xpath.yml file that contains xpath's to scrape"""

    with open("xpath.yml", "r") as f:
        content:dict[str:str] = yaml.safe_load(f)
        return content[name]

In [6]:
def close_prompt(driver:object) -> None:
    """Close ad window"""

    button:str = "//a[@class='btn btn-subscription-light']"
    driver.find_element(by=By.XPATH, value=button).click()

In [7]:
def choose_category(category:str, driver:object) -> None:
    """Choose ad category"""

    driver.find_element(by=By.XPATH, value=category).click()

In [8]:
def click_element(element:object, path:str) -> None:
    """Click element on the current document"""

    button = element.find_element(by=By.XPATH, value=path)
    button.click()

In [9]:
def extract_description(element:object, path:str) -> str:
    """Extract ad description on the current document"""

    description:str = element.find_element(by=By.XPATH, value=path)
    return description

In [10]:
def extract_feature(element:object, path:str) -> list[str]:
    """Extract some feature from list document based on xpath used."""

    feature = []

    try:
        feature:str = element.find_elements(by=By.XPATH, value=path)
        feature = [data_point.text for data_point in feature]
        return feature
    except Exception:
        feature.append("Web wasn't able to scrape the feature")

In [11]:
def persistence(dataframe:pd.DataFrame) -> None:
    """Generate .csv file from data scraped"""

    file_name = datetime.today().strftime("%B-%d")
    dataframe.to_csv(f"persistence/{file_name}.csv", index=False)

In [12]:
def get_len(features:list[list[str]]) -> bool:
    """
    Check if all features has the same length to create new df
    :param features: list of features extracted with scraping
    :return: a list with all lengths where first place is the len of the first element and the mostly accepted.
    """

    base_len = len(features[0])
    lengths = []
    lengths.append(base_len)
    for feature in features:
        if len(feature) == base_len:
            continue
        else:
             lengths.append(len(feature))
    return lengths

In [13]:
def check_len(lengths:list[int]) -> bool:
    """
    Check if all elements in list has the same value.
    :param lengths: list of lengths
    :return: a boolean
    """
    for length in lengths[1:]:
        if length != lengths[0]:
            return False
        return True

Variables pool.

In [14]:
descriptions:list[str] = []
prices:list[int] = []
models:list[str] = []
years:list[int] = []
kilometers:list[int] = []
engines:list[str] = []

# seconds
DELAY:int = 1

In [15]:
sedan = resources("cars_sedan")

Get web & close initial prompt.

In [16]:
edge.get(sedan["url"])

In [17]:
# close_prompt(edge)

In [18]:
elements = edge.find_elements(by=By.XPATH, value=sedan["ad"])

Extract descriptions.

In [19]:
elements[0].click()
try:
    for element in range(len(elements)):
        dynamic_delay = WebDriverWait(edge, DELAY).until(EC.presence_of_element_located((By.XPATH, sedan["description"])))
        temp = extract_description(edge, sedan["description"])
        descriptions.append(temp.text)
        click_element(edge, sedan["next"])
except TimeoutException as TE:
    descriptions.append("Web wasn't able to scrape descriptions")
except NoSuchElementException as NSE:
    click_element(edge, sedan["exit"])

Extract prices & models.

In [20]:
prices = extract_feature(edge, sedan["price"])
models = extract_feature(edge, sedan["model"])

Extract car year, engine & kilometers.

In [21]:
years = extract_feature(edge, sedan["year"])
engines = extract_feature(edge, sedan["engine"])
kilometers = extract_feature(edge, sedan["kilometers"])

In [22]:
lengths = [prices, models, years, engines, kilometers, descriptions]
get_len(lengths)

[40, 41, 41, 37]

In [42]:
kilometers.pop(35)

'Gasolina'

In [43]:
kilometers.index("Gasolina")

ValueError: 'Gasolina' is not in list

In [41]:
kilometers.insert(34, "0 km")

## Export

---

### Functions:

In [23]:
def parse_price(dataframe: pd.DataFrame, feature:str, pattern:str) -> None:
    """
    Parse financial features like prices. Works with $#,### or $#,###\n* pattern
    :param dataframe: object towards apply transformation
    :param feature: feature name towards apply transformation
    :param pattern: feature pattern to transform
    """
    dataframe[feature] = dataframe [feature] \
        .str.extract(pattern) \
        .applymap(lambda price: int(price.replace(",","")))


In [24]:
def remove_chars(dataframe: pd.DataFrame, feature:str, wildcards:str) -> pd.Series:
    """
    Remove escaped characters from features like indented large extracts of text
    :param dataframe: object towards apply transformation
    :param feature: feature name towards apply transformation
    :param wildcards: escaped characters to remove
    :return: Serie without escaped characters
    """
    return (dataframe
    .apply(
        lambda row: row[feature]
        ,axis = 1
    )
    .apply(
        lambda extracted_f: re.sub(wildcards, r' ', extracted_f)
    )
    )


In [25]:
PRICE_RG: str = r"^\$(?P<prices>\d+,\d+)?"
KM_REG: str = r"(?P<km>^\d+)"
CHARACTER_RG:str = '(\n|\r)+'

Remove noise from `model` and `years` lists

In [26]:
years.remove("Ver más")

In [27]:
models.remove("Blue Book Encuentra24")

In [44]:
lengths = [len(engines), len(models), len(years), len(kilometers), len(prices),len(descriptions)]
lengths

[40, 40, 40, 40, 40, 40]

Create dataframe to transform data extracted

In [45]:
if check_len(lengths):

    df_temp = pd.DataFrame({
        "model": models,
        "year": years,
        "kilometers": kilometers,
        "engine": engines,
        "price": prices,
        "description": descriptions
    })

    # Parse features. String to number and raw string with characters to structured string.
    parse_price(df_temp, "price", PRICE_RG)
    df_temp["kilometers"] = df_temp["kilometers"].str.extract(KM_REG).applymap(int)
    df_temp["year"] = df_temp["year"].map(int)
    df_temp["description"] = remove_chars(df_temp, "description", CHARACTER_RG)

    persistence(df_temp)


In [46]:
df_temp

,model,year,kilometers,engine,price,description
0,Nissan Versa,2017,78000,Gasolina,7100,Color : Azul DESCRIPCIÓN CALIFICACIONES SEGURI...
1,Hyundai Elantra,2000,204000,Gasolina,2450,Garantía: Como es visto no hay garantía Financ...
2,Kia Rio,2016,130,Gasolina,7500,Precio negociable: si Color : Cafe DESCRIPCIÓN...
3,Nissan Versa,2016,99000,Gasolina,6800,Precio negociable: si Color : Negro DESCRIPCIÓ...
4,Chevrolet Spark,2020,83903,Gasolina,7350,Garantía: Como es visto no hay garantía Financ...
5,BMW 328,2012,90000,Gasolina,10500,Financiamiento: no Precio negociable: si Color...
6,Kia Forte,2015,72000,Gasolina,7750,Garantía: Como es visto no hay garantía Financ...
7,Nissan Versa,2017,86104,Gasolina,7000,Precio negociable: si DESCRIPCIÓN CALIFICACION...
8,Hyundai Accent,2017,96000,Gasolina,8200,Color : Negro DESCRIPCIÓN SEGURIDAD Hyundai ac...
9,Honda Civic,2010,208355,Gasolina,6600,Color : Negro DESCRIPCIÓN CALIFICACIONES SEGUR...
